# Chapter 4
## SOLID 원칙
- S : 단일 책임 원칙
- O : 개방 / 폐쇄의 원칙
- L : 리스코프(Liskov)치환 원칙
- I : 인터페이스 분리 원칙
- D : 의존성 역전 원칙

## 목표
- 소프트웨어 디자인에서의 SOLID원칙을 익힌다.
- 단일 책임 원칙을 따르는 컴포넌트를 디자인한다.
- 개방 / 폐쇄의 원칙을 통해 보다 나은 유지보수성을 얻는다.
- 리스코프 치환 원칙을 준수하여 객체지향 디자인에서 적절한 클래스 계층을 설계한다.
- 인터페이스 분리와 의존성 역전을 활용해 설계하기

## Single Responsibility Principle - SRP (단일 책임 원칙)
- 소프트웨어 component (일반적으로 class)는 단 하나의 책임을 져야한다는 원칙이다.
- 만약 class를 수정해야한다면 단 하나의 이유만이 존재해야한다.
- 다른 이유가 존재한다면 class가 너무 많은 책임을 지고 있다는 것이다.

- 너무 많은 책임을 가진 객체를 GOD 객체라고 한다.
- class는 작을 수록 좋다.
- class와 method는 상호 배타적이여야 하며 서로 관련이 없어야한다.

### 너무 많은 책임을 가진 클래스

In [1]:
class SystemMonitor:
    def load_activity(self):
        """소스에서 처리할 이벤트를 가져오기"""

    def identify_events(self):
        """가져온 데이터를 파싱하여 도메인 객체 이벤트로 변환"""

    def stream_events(self):
        """파싱한 이벤트를 외부 에이전트로 전송"""

- 위 클래스의 문제점은 독립적인 동작을 하는 메서드를 하나의 인터페이스에 정의했다는 것이다.
- 예를 들어 loader를 생각해보자 data의 형식이 바뀌어 loader class를 바꿔야하는데<br>
loader함수가 SystemMonitor class에 정의 돼있으므로 data 형식의 변환이 SystemMonitor<br>
전체를 수정하게 한다. 이는 좋지않다.
- 아래 2개의 경우에도 마찬가지이며, 해결책은 보다 작고 응집력이 강한 객체로 추상화하는 것이다.

## 책임 분산
- 솔루션을 보다 관리하기 쉽게 하기 위해 모든 메서드를 다른 클래스로 분리하여 각 클래스마다<br>
단일 책임을 갖게 하자!

- 각자의 책임을 가진 여러 객체들을 만들고 이 객체들이 협력하여 동일한 기능을 수행하는 객체를 만들 수 있다.
- app의 다른 부분에서 로그를 다른 용도로 읽어야 한다고 생각해보자. 이때 책의 115.p 를 보면<br>
ActivityReader 타입의 객체를 사용하면 된다.

## Open / Close Principle (개방 / 폐쇄 원칙)
- 위 원칙은 모듈이 개방되어 있으면서도 폐쇄되어야 한다는 원칙이다.
- 클래스를 디자인할 때는 유지보수가 쉽도록 로직을 캡슐화하여 확장에는 개방되고 수정에는 폐쇄되도록 해야한다.

- 새로운 요구사항이나 도메인 변화에 잘 적응하는 코드를 작성해야 한다는 뜻이다.
- 즉, 새로운 문제가 발생할 경우 새로운 것을 추가만 할 뿐 기존 코드는 그대로 유지되어야 한다.

### Open / Close Principle 을 따르지 않을 경우 유지보수의 어려움

In [2]:
# class Event:
#     def __init__(self, raw_data):
#         self.raw_data = raw_data
#
# class UnknownEvent(Event):
#     '''데이터만으로 식별할 수 없는 이벤트'''
#
# class LoginEvent(Event):
#     '''로그인 사용자에 의한 이벤트'''
#
# class LogoutEvent(Event):
#     '''로그아웃 사용자에 의한 이벤트'''
#
# class SystemMonitor:
#     '''시스템에서 발생한 이벤트 분류'''
#
#     def __init__(self, event_data):
#         self.event_data = event_data
#
#     def identify_event(self):
#         if(
#             self.event_data['before']['session'] == 0 and
#             self.event_data['after']['session'] == 1
#         ):
#             return LoginEvent(self.event_data)
#         elif(
#             self.event_data['before']['session'] == 1 and
#             self.event_data['after']['session'] == 0
#         ):
#             return LogoutEvent(self.event_data)
#
#         return UnknownEvent(self.event_data)

위 코드는 아래와 같이 동작한다.

In [3]:
l1 = SystemMonitor({'before': {'session': 0}, 'after': {'session': 1}})
l1.identify_event().__class__.__name__

TypeError: SystemMonitor() takes no arguments

In [ ]:
l2 = SystemMonitor({'before': {'session': 1}, 'after': {'session': 0}})
l2.identify_event().__class__.__name__

In [ ]:
l3 = SystemMonitor({'before': {'session': 1}, 'after': {'session': 1}})
l3.identify_event().__class__.__name__

이벤트 유형의 계층 구조와 이를 구성하는 로직을 명확히 알 수 있다.<br>

- 이 디자인에는 몇 가지 문제점이 있다.

이벤트 유형을 결정하는 로직이 일체형으로 집중화된다는 점이다.<br>
지원하려는 이벤트가 늘어날  수록 메서드도 커지므로 결국엔 엄청나게 커질 수 있다.<br>

- 같은 방법으로 이 메서드가 수정을 위해 닫히지 않았다는 것을 알 수 있다.
- 새로운 유형의 이벤트를 시스템에 추가할 때마다 메서드를 수정해야한다. elif문은 최악이다..!

### 이 메서드를 변경하지 않고도 새로운 유형의 이벤트를 추가하고 싶다. (폐쇄 원칙)
### 새로운 이벤트가 추가될 때 이미 존재하는 코드를 변경하지 앟고 코드를 확장하여<br>
### 새로운 유형의 이벤트를 지원하고 싶다. (개방 원칙)

- 이전 예제의 문제점은 매번 구체 클래스와 직접 상호 작용 한다는 점이다.
- 개방 폐쇄 원칙을 따르는 디자인을 하려면 추상화를 해야한다.

SystemMonitor 는 추상적인 이벤트와 협력하도록 변경하고<br>
이벤트에 대응하는 개별 로직은 각 이벤트를 클래스에 위임하는 것이다.

In [ ]:
# class Event:
#     def __init__(self, raw_data):
#         self.raw_data = raw_data
#
#     @staticmethod
#     def meets_condition(event_data: dict):
#         return False
#
# class UnknownEvent(Event):
#     '''데이터만으로 식별할 수 없는 이벤트'''
#
# class LogicEvent(Event):
#     @staticmethod
#     def meets_condition(event_data: dict):
#         return (
#             event_data['before']['session'] == 0 and
#             event_data['after']['session'] == 1
#         )
#
# class LogoutEvent(Event):
#     @staticmethod
#     def meets_condition(event_data: dict):
#         return (
#             event_data['before']['session'] == 1 and
#             event_data['after']['session'] == 0
#         )
#
# class SystemMonitor:
#     '''시스템에서 발생한 이벤트 분류'''
#
#     def __init__(self, event_data):
#         self.event_data = event_data
#
#     def identify_event(self):
#         for event_cls in Event.__subclasses__():
#             try:
#                 if event_cls.meets_condition(self.event_data):
#                     return event_cls(self.event_data)
#
#             except KeyError:
#                 continue
#
#         return UnknownEvent(self.event_data)

- 이제 상호 작용이 추상화를 통해 이뤄지고 있다.
- 분류 메서드는 이제 특정 이벤트 타입 대신에 일반적인 인터페이스를 다르는 제네릭 이벤트와 동작한다.
- 이 인터페이스를 따르는 제네릭들은 모두 meets_condition 메서드를 구현하여 다형성을 보장한다.

- \_\_subclasses__() 메서드를 사용해 이벤트 유형을 찾는 것에 주목하자.
- 이제 새로운 유형의 이벤트를 지원하려면 단지 Event 클래스를 상속 받아 로직에 다라 meets_condition()  메서드를 구현하기만 하면 된다.

- 아래와 같이 새로운 이벤트를 추가해보자

In [ ]:
# class TransactionEvent(Event):
#     '''시스템에서 발생한 트랜잭션 이벤트'''
#
#     @staticmethod
#     def meets_condition(event_data: dict):
#         return event_data['after'].get('transaction') is not None

In [ ]:
# l1 = SystemMonitor({'before': {'session': 0}, 'after': {'session': 1}})
# l1.identify_event().__class__.__name__

In [ ]:
# l2 = SystemMonitor({'before': {'session': 1}, 'after': {'session': 0}})
# l2.identify_event().__class__.__name__

In [ ]:
# l3 = SystemMonitor({'before': {'session': 1}, 'after': {'session': 1}})
# l3.identify_event().__class__.__name__

In [ ]:
# l4 = SystemMonitor({'after': {'transaction': 'Tx001'}})
# l4.identify_event().__class__.__name__

- 이것이 바로 open / Close Principle 이다.

- OCP는 다형성의 효과적인 사용과 밀접하게 관련이 있다.
- 물론 상황에 따라 다르게 추상화 해야하므로 적합한 선택을 해야한다.

## Liskov 치환 원칙 (LSP)
### 설계 시 안정성을 유지하기 위해 객체 타입이 유지해야하는 일련의 특성을 말한다.

- 어떤 클래스에서든 클라이언트는 특별한 주의를 기울이지 않고도 하위 타입을 사용할 수 있어야한다.
- 즉, 클라이언트는 완전히 분리되어 있으며 클래스 변경 사항과 독립되어야 한다.

- 만약 S가 T의 하위 타입이라면 프로그램을 변경하지 않고 T 타입의 객체를 S 타입의 객체로 치환 가능해야 한다.
- 계층 구조가 올바르게 구현되었다면 클라이언트 클래스가 주의를 기울이지 않고도 모든 하위 클래스의 인스턴스로 작업할 수 있어야한다.
- LSP의 규칙에 따르면 하위 클래스는 상위 클래스에서 정의한 계약을 따르도록 디자인 해야한다.

### 도구를 사용해 LSP 문제 검사하기

- Mypy나 Pylint 같은 도구를 사용해 쉽게 검출할 수 있다.

In [ ]:
# class Event:
#     def __init__(self, raw_data):
#         self.raw_data = raw_data
#
#     @staticmethod
#     def meets_condition() -> bool:
#         return False
#
# class LoginEvent(Event):
#     @staticmethod
#     def meets_condition(event_data: list) -> bool:
#         return bool(event_data)

- meets_condition() 메서드를 보면 signature가 다르다.
- LSP 원칙에 따르면 호출자는 아무런 차이를 느끼지 않고 투명하게 Event 또는 LoginEvent를 사용할 수 있어야한다.
- 이 두 가지 타입의 객체를 치환해도 애플리케이션 실행에 실패해서는 안된다. 그렇지 않으면 계층 구조의 다형성이 손상된 것이다.

- 이번 예제는 사전조건에서 파라미터가 dict인지, 'before', 'after'키를 가지고 있는지 확인한다.
- 'before', 'after' 키의 값은 또다시 객체를 내포해야한다.
- 이렇게 하면 클라이언트는 KeyError를 받지 않으므로 보다 발전된 캡슐화를 할 수 있다.

- 다음과 같이 위 코드를 수정할 수 있다.

In [ ]:
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False

    @staticmethod
    def meets_condition_pre(event_data: dict):
        """
        인터페이스 계약의 사전조건
        event_data 파라미터가 적절한 형태인지 유효성 검사
        """

        assert isinstance(event_data, dict), f'{event_data!r} is not a dict'
        for moment in ('before', 'after'):
            assert moment in event_data, f'{moment} not in {event_data}'
            assert isinstance(event_data[moment], dict)

class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        Event.meets_condition_pre(self.event_data)
        event_cls = next(
            (
                event_cls
                for event_cls in Event.__subclasses__()
                if event_cls.meets_condition()
            )
            # ? UnknownEvent
        )
class TransactionEvent(Event):
    """시스템에서  발생한 이벤트 분류"""

    @staticmethod
    def meets_condition(event_data: dict):
        return event_data['after'].get('transaction') is not None

## Interface Segregation Principle (ISP)

- 인터페이스는 객체가 노출하는 메서드의 집합이다.
- 파이썬에서 인터페이스는 클래스 메서드의 형채를 보고 암시적으로 정의된다.
- 이것은 소위 말하는 duck typing 원리를 따르기 때문이다.
- "어떤 새가 오리처럼 걷고 오리처럼 꽥꽥 소리를 낸다면 오리여야만 한다." 는데서 duck typing 이라고 한다.

- 추상 깁존 클래스는 파생 클래스가 구현해야 할 일부분을 기본 동작 또는 인터페이스로 정의하는 것이다.

- ISP는 다중 메서드를 가진 인터페이스가 있다면 매우 정확하고 구체적인 구분에 따라 더 적은 수의
- 메서드를 가진 여러 개의 메서르도 분할 하는 것이 좋다는 것이다.
- 재사용성을 높이기 위해 가능한 작은 단위로 인터페이스를 분리한다면 인터페이스 중 하나를 구현하려는
- 각 클래스가 매우 명확한 동작과 책임을 지닌다.

### 인터페이스는 작을수록 좋다.

## 의존성 역전 (DIP)
- 코드가 깨지거나 손상되는 취약점으로부터 보호해주는 흥미로운 디자인 원칙을 제시한다.
- 의존성을 역전 시킨다는 것은 코드가 세부 사항이나 구체적인 구현에 적응 하도록 하지 않고, API 같은 것에 적응하도록 하는 것이다.

- A, B 모듈이 상호 교류 할 때 하나의 모듈이 다른 모듈에 너무 의존적이라면 다른 모듈에서 수정이 있을 경우
- 남은 모듈이 쉽게 깨질 수 있다. 따라서 의존성을 역전시킬 필요가 있다.

- 위와 같이 하기 위해 정의한 인터페이스에 의존적이도록 해야한다.
- 일반적으로 구체적인 구현이 추상 컴포넌트보다 훨씬 더 자주 바뀔 것이다. 이런 이유로 시스템이 변경, 수정
- 또는 확장될 것으로 예상되는 지점에 유연성을 확보하기 위해 추상화(인터페이스 사용)를 하는 것이다.

### 엄격한 의존 예시

- 이벤트 모니터일 시스템의 마지막 부분은 식별된 이벤트를 데이터 수집기로 전달하여 분석하는 것이었다.
- 그러나 이것은 저수준의 내용에 따라 고수준의 클래스가 변경되어야 하므로 좋은 디자인이 아니다.

### 의존성을 거꾸로
- 이러한 문제를 해결하려면 구체 클래스가 아닌 인터페이스와 대화하도록 하는 것이 좋다.
- 심지어 런타임 중에도 send() 메서드를 구현한 객체의 프로퍼티를 수정해도 여전히 잘 동작한다.
- 이렇게 동적으로 제공한다고 하여 종종 의존성 주입(dependency injection)이라고 한다.

- 상속은 'A is a B' 라는 관계를 잊지말자.
- 파이썬은 유연하여 인터페이스를 꼭 사용하지 않아도 좋지만
- 인터페이스를 사용하는 것은 좋은 습관이다.







































